In [1]:
from utils.get_data import generate_data_nested_dirs, get_cube_at_point, generate_cubes, generate_cube_batch
import numpy as np

In [2]:
import pandas as pd

df_annotations = pd.read_csv('CSVFILES/annotations.csv')

In [3]:
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dense, Dropout, ReLU

In [52]:
train_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 16,
    rotate=30,
    axes=(0,1,2),
    offset=0.25
)

test_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/test',
    batch_size = 16
)

val_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/val',
    batch_size = 16
)

In [54]:
model = Sequential()
# model.add(Conv3D(
#         filters = 32,
#         kernel_size = (5,5,5),
#         strides = (1,1,1),
#         padding = 'valid',
#         activation = 'relu',
#         use_bias = True,
#         kernel_initializer = 'glorot_uniform',
#         bias_initializer = 'zeros',
#         kernel_regularizer = None,
#         bias_regularizer = None,
#         activity_regularizer = None,
#         kernel_constraint = None,
#         bias_constraint = None,
#         input_shape = (25,64,64,1)))
# model.add(MaxPooling3D(
#         pool_size = (2,2,2),
#         strides = None,
#         padding = 'valid'))
# model.add(BatchNormalization(axis = 1))
# model.add(Dropout(0.2))
model.add(Flatten(input_shape=(25,64,64,1)))
# model.add(Dense(
#         units = 256,
#         activation = 'relu',
#         use_bias = True,
#         kernel_initializer = 'glorot_uniform',
#         bias_initializer = 'zeros',
#         kernel_regularizer = None,
#         bias_regularizer = None,
#         activity_regularizer = None,
#         kernel_constraint = None,
#         bias_constraint = None))
model.add(Dense(
        units = 32,
        activation = 'relu',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.add(Dense(
        units = 2,
        activation = 'softmax',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3276832   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 3,276,898
Trainable params: 3,276,898
Non-trainable params: 0
_________________________________________________________________


In [55]:
sgd = optimizers.SGD(
    lr=0.0001, 
    decay = 1e-4, 
    nesterov=False
)

model.compile(
    optimizer=sgd,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x = train_generator,
    validation_data = val_generator,
    validation_steps = 25,
    epochs = 2,
    shuffle=True,
    steps_per_epoch = 75
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 75 steps, validate for 25 steps
Epoch 1/2
75/75 [==============================] - 1205s 16s/step - loss: 232521.1927 - accuracy: 0.6167 - val_loss: 75610.3816 - val_accuracy: 0.6175
Epoch 2/2
74/75 [============================>.] - ETA: 11s - loss: 50710.8685 - accuracy: 0.6546

In [ ]:
model.save('first_model.h5')

In [48]:
y_pred = model.predict(train_generator, steps = 25)

In [49]:
y_pred = np.argmax(y_pred, axis=1)

In [11]:
from sklearn.metrics import confusion_matrix

In [50]:
test_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 16
)

y = np.zeros((1,2))
for i in range(25):
    x, batch = next(test_generator)
    if not y.any():
        y = np.argmax(batch, axis=1)
        continue
    y = np.concatenate((y, np.argmax(batch, axis=1)))

In [45]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [51]:
confusion_matrix(y,y_pred)

array([[195,   1],
       [  4, 200]])